In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/useriteminteractionspoetry/interactions_poetry.csv
/kaggle/input/goodreads-interactions-poetry/goodreads_interactions_poetry.json
/kaggle/input/goodbooks-books-poetry/goodreads_books_poetry.json


In [2]:
!pip install similaripy
!pip install scikit-surprise

     |████████████████████████████████| 334 kB 6.3 MB/s eta 0:00:01
  Created wheel for similaripy: filename=similaripy-0.1.1-cp37-cp37m-linux_x86_64.whl size=1515867 sha256=65602ff3fa0ab857468aff5773c3082ebcd987cdb5248a53f6810c68f1e970e5
  Stored in directory: /root/.cache/pip/wheels/52/77/1c/8da70f5b02be559b78118a9867c2fd864ed1d221d3d440ee95
Successfully built similaripy


In [14]:
import json
from pandas.io.json import json_normalize
from sys import getsizeof
from sklearn.metrics import mean_squared_error, mean_absolute_error

from scipy.sparse import csr_matrix, csc_matrix, coo_matrix
import scipy.sparse as sp
from scipy import *

import similaripy as sim
from similaripy.normalization import normalize, bm25, bm25plus, tfidf

from implicit.nearest_neighbours import ItemItemRecommender, bm25_weight, tfidf_weight
import implicit

from surprise import SVD
from surprise import SVDpp
from surprise.model_selection import cross_validate
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split

# For displaying images
from IPython.display import Image 
from matplotlib.pyplot import figure, imshow, axis
from matplotlib.image import imread


## Importing Data

The following two cells are temporary, and will be removed once we have the interactions and book data in the required formats

In [4]:
interactions_df = pd.read_csv('/kaggle/input/useriteminteractionspoetry/interactions_poetry.csv')
display(interactions_df.head(5))
display(interactions_df.describe())
#display(interactions_df.info)

,Unnamed: 0,user_id,book_id,review_id,is_read,rating,review_text_incomplete,date_added,date_updated,read_at,started_at
0,0,8842281e1d1347389f2ab93d60773d4d,1384,1bad0122cebb4aa9213f9fe1aa281f66,True,4,NaN,Wed May 09 09:33:44 -0700 2007,Wed May 09 09:33:44 -0700 2007,NaN,NaN
1,1,8842281e1d1347389f2ab93d60773d4d,1376,eb6e502d0c04d57b43a5a02c21b64ab4,True,4,NaN,Wed May 09 09:33:18 -0700 2007,Wed May 09 09:33:18 -0700 2007,NaN,NaN
2,2,8842281e1d1347389f2ab93d60773d4d,30119,787564bef16cb1f43e0f641ab59d25b7,True,5,NaN,Sat Jan 13 13:44:20 -0800 2007,Wed Mar 22 11:45:08 -0700 2017,Tue Mar 01 00:00:00 -0800 1983,NaN
3,3,72fb0d0087d28c832f15776b0d936598,24769928,8c80ee74743d4b3b123dd1a2e0c0bcac,False,0,NaN,Wed Apr 27 11:05:51 -0700 2016,Wed Apr 27 11:05:52 -0700 2016,NaN,NaN
4,4,72fb0d0087d28c832f15776b0d936598,30119,2a83589fb597309934ec9b1db5876aaf,True,3,NaN,Mon Jun 04 18:58:08 -0700 2012,Mon Jun 04 18:58:13 -0700 2012,NaN,NaN


,Unnamed: 0,book_id,rating
count,2.734350e+06,2.734350e+06,2.734350e+06
mean,1.367174e+06,6.808744e+06,1.824787e+00
std,7.893390e+05,9.698381e+06,2.123223e+00
min,0.000000e+00,2.340000e+02,0.000000e+00
25%,6.835872e+05,4.204000e+04,0.000000e+00
50%,1.367174e+06,5.922210e+05,0.000000e+00
75%,2.050762e+06,1.219330e+07,4.000000e+00
max,2.734349e+06,3.648548e+07,5.000000e+00


In [8]:
interactions_df.tail()

,user_id,book_id,review_id,is_read,rating,review_text_incomplete,date_added,date_updated,read_at,started_at
2734345,594c86711bd7acdaf655d102df52a9cb,10810038,1b6982ab9342de8d0c560f74598b7b95,False,0,NaN,Thu May 23 12:54:29 -0700 2013,Thu May 23 12:54:29 -0700 2013,NaN,NaN
2734346,594c86711bd7acdaf655d102df52a9cb,4488657,6fccf41aad2ef9d4392b2dc0c14cf77c,False,0,NaN,Sun Apr 28 09:26:58 -0700 2013,Sun Apr 28 09:27:11 -0700 2013,NaN,NaN
2734347,594c86711bd7acdaf655d102df52a9cb,5865674,dea3556a048fc6b75f361baeaa0c49e9,False,0,NaN,Sun Apr 28 09:25:28 -0700 2013,Sun Apr 28 09:25:28 -0700 2013,NaN,NaN
2734348,594c86711bd7acdaf655d102df52a9cb,16170625,a3d153de0b86bdf9eeef0d756f471649,True,5,NaN,Sun Apr 21 16:29:46 -0700 2013,Fri May 31 10:29:35 -0700 2013,NaN,NaN
2734349,594c86711bd7acdaf655d102df52a9cb,16101638,ca49618c80bf2111ce717d1d795a74d9,True,4,NaN,Sat Apr 20 15:29:12 -0700 2013,Sun Apr 21 14:21:37 -0700 2013,NaN,NaN


In [9]:
interactions_df.head()

,user_id,book_id,review_id,is_read,rating,review_text_incomplete,date_added,date_updated,read_at,started_at
0,8842281e1d1347389f2ab93d60773d4d,1384,1bad0122cebb4aa9213f9fe1aa281f66,True,4,NaN,Wed May 09 09:33:44 -0700 2007,Wed May 09 09:33:44 -0700 2007,NaN,NaN
1,8842281e1d1347389f2ab93d60773d4d,1376,eb6e502d0c04d57b43a5a02c21b64ab4,True,4,NaN,Wed May 09 09:33:18 -0700 2007,Wed May 09 09:33:18 -0700 2007,NaN,NaN
2,8842281e1d1347389f2ab93d60773d4d,30119,787564bef16cb1f43e0f641ab59d25b7,True,5,NaN,Sat Jan 13 13:44:20 -0800 2007,Wed Mar 22 11:45:08 -0700 2017,Tue Mar 01 00:00:00 -0800 1983,NaN
3,72fb0d0087d28c832f15776b0d936598,24769928,8c80ee74743d4b3b123dd1a2e0c0bcac,False,0,NaN,Wed Apr 27 11:05:51 -0700 2016,Wed Apr 27 11:05:52 -0700 2016,NaN,NaN
4,72fb0d0087d28c832f15776b0d936598,30119,2a83589fb597309934ec9b1db5876aaf,True,3,NaN,Mon Jun 04 18:58:08 -0700 2012,Mon Jun 04 18:58:13 -0700 2012,NaN,NaN


In [10]:
# In this cell, I'll clean all the data, and make sure there aren't any anomalies


# interactions_df.drop(['Unnamed: 0'], axis = 1, inplace = True)
interactions_df = interactions_df.drop(columns = ['review_id','is_read','review_text_incomplete','date_added','date_updated','read_at','started_at'])
display(interactions_df.head(5))



,user_id,book_id,rating
0,8842281e1d1347389f2ab93d60773d4d,1384,4
1,8842281e1d1347389f2ab93d60773d4d,1376,4
2,8842281e1d1347389f2ab93d60773d4d,30119,5
3,72fb0d0087d28c832f15776b0d936598,24769928,0
4,72fb0d0087d28c832f15776b0d936598,30119,3


In [11]:
# I still have to remove all the reviews with 0 ratings. Becuase, they haven't actually read the book
interactions_df = interactions_df[interactions_df.rating != 0]
display(interactions_df.head(5))

,user_id,book_id,rating
0,8842281e1d1347389f2ab93d60773d4d,1384,4
1,8842281e1d1347389f2ab93d60773d4d,1376,4
2,8842281e1d1347389f2ab93d60773d4d,30119,5
4,72fb0d0087d28c832f15776b0d936598,30119,3
5,ab2923b738ea3082f5f3efcbbfacb218,240007,4


In [12]:
interactions_df['user_idInt']= interactions_df['user_id'].str.replace('\D+','').astype(float)
interactions_df = interactions_df.drop(columns = ['user_id'])
display(interactions_df)

,book_id,rating,user_idInt
0,1384,4,8.842281e+23
1,1376,4,8.842281e+23
2,30119,5,8.842281e+23
4,30119,3,7.200087e+23
5,240007,4,2.923738e+15
...,...,...,...
2734340,2547,5,1.484971e+21
2734341,11047097,4,6.951628e+21
2734343,7433930,5,5.948671e+17
2734348,16170625,5,5.948671e+17


### Supporting Functions
Here I'll try to convert the entire data in form of scipy.sparse matrices, and download that data using scipy.sparse.save_npz(). This way, perhaps, our memory usage would be a bit lesser, and we could fit more of the data in the 16 Gigs we have at our disposal.

In [ ]:

def showImagesHorizontally(list_of_files):
    '''
    This function shows all the books in a horizontal row within the ipython notebook
    '''
    fig = figure()
    number_of_files = len(list_of_files)
    for i in range(number_of_files):
        a=fig.add_subplot(1,number_of_files,i+1)
        image = imread(list_of_files[i])
        imshow(image)
        axis('off')

        
def print_book_images(book_df, recommendations):
    '''
    This function will display the images along with Titles and author names of all the books recommended 
    Args:
        recommendations (iterable) -> Containing the book_ids of the recommendations made by the recommender system
        book_df (DataFrame) -> DataFrame containing basic info of all the books part of our product ('book_id','url','image_url','authors', 'title')
        
        
    '''
    #import requests
    #import IPython.display as Disp
    #url = 'https://upload.wikimedia.org/wikipedia/commons/5/56/Kosaciec_szczecinkowaty_Iris_setosa.jpg'
    #Disp.Image(requests.get(url).content)
    temp_df = book_df.loc[book_df['book_id'] == recommendations]
    width_pixels = 200
    height_pixels = 300
    book_urls = []
    for id in recommendations:
        book = book_df.loc[book_df['book_id'] == id]
        image_url = book['image_url']
        book_urls.append(image_url)
        title = book['title']
        authors = book['authors']
        Image(image_url,width=width_pixels, height=height_pixels)
        print('Title : ', title)
        print('Authors : ', authors)
        width_pixels = width_pixel - 10
        height_pixels = height_pixels - 8
    
    showImagesHorizontally(book_urls)
    
    
    
def print_book_ratings(data):
    '''
    This function will print the book ratings of the top-k book recommendations passed along to it
    
    '''
    

#### Functions for Downloading the sparce matrices

In [ ]:
def create_sparse_matrix(data, user_dict, item_dict):
    '''
    This function creates a csr sparse matrix from the given data. Data should be a Dataframe (preferably derived from a .csv file)
    
    Args:
        data (DataFrame) -> Containing book_id, user_idInt, and rating for each instance
        user_dict(Pandas series) -> Contains the unique set of user ids present within the data
        item_dict (Pandas Series) -> Contains the unique set of items present in the data
    Returns:
        csr_item_matrix (sparse csr matrix) -> csr item matrix created from the data
        user_dict, item_dict -> Both updated
    '''
    # Here I'll create 'user_dict' & 'item_dict' such that: the indexes of the array would represent the "key" for the dicts.
    temp_user_dict = pd.concat([user_dict, pd.unique(data['user_idInt'])], ignore_index = True)
    temp_item_dict = pd.append(pd.unique(data['book_id']), ignore_index = True)
    user_dict = pd.unique(temp_user_dict)
    item_dict = pd.unique(temp_item_dict)

    # Then, implement the csr_mechanism and save the result in self.csr_item_matrix
    rows = []
    cols = []
    data = []
    for i in range(len(data)):
        rows.append(user_dict.index(data.loc[i, 'user_idInt'])) # Here we creating the row values for csr matrix using the index values of the user_dict we created by mapping unique values in a dataframe
        cols.append(item_dict.index(data.loc[i, 'book_id'])) # Maybe, the index part should be in square brackets as well... Will see
        data.append(data.loc[i, 'rating'])
    rows = np.array(rows)
    cols = np.array(cols)
    data = np.array(data)
    csr_item_matrix = csr_matrix(data, (row, col))
    return csr_item_matrix.tocsr(), user_dict, item_dict


In [ ]:
# Here I'll create the sparse matrix using create_sparse_matrix function and then download it. Following which I'll upload it directly, and see if I can save further memory space this way
def download_sparse_matrix(filename, csr_matrix):
    '''
    This function will download the sparse matrix. I will be calling this function multiple times to convert and store all the interactions data in this very format.
    
    Actually, I might not have to make a seperate npz file for each genre data, instead, I can create a single npz file with multiple (compressed) chunks of data from 
    all the genres. The good thing about npz format is that it allows for multiple files to be stored such that only the ones being used are loaded onto the memory.
    Checkout -https://stackoverflow.com/questions/54238670/what-is-the-advantage-of-saving-npz-files-instead-of-npy-in-python-regard
    
    
    '''
    sp.save_npz(filename, matrix)
    

#### Implementation code for the supporting functions

Here, we'll be calling all the supporting functions

In [ ]:
# Here, I will clean the interactions data (removing unnecessary columns: maybe this can be done in a function, so that different data can be produced for implicit and explicit models) & call the two functions above. 

# Create_sparse_Matrix will probably have to be called for each genre or chunk we want, while (hopefully) download sparse can be called once.
#  I will also have to look into how to add files to .npz format. Cos, maybe I can't load all the json data and convert it into npy files in one go.


# Create user_dict and item_dict (to maintain uniqueness across varioud datasets)
user_dict = pd.DataFrame()
item_dict = pd.DataFrame()
book_df = pd.DataFrame(columns = ['book_id','url','image_url','authors', 'title'])
# Perform the following code for each of the genre, and make sure user_dict and item_dict are maintained along all of them

# Convert data and add it in form of lists
interactions_data = []
with open("/kaggle/input/goodreads-interactions-poetry/goodreads_interactions_poetry.json", 'r') as f:
    for line in f:
        interaction_data.append(json.loads(line))

books_data = []
with open("/kaggle/input/goodbooks-books-poetry/goodreads_books_poetry.json", 'r') as f:
    for line in f:
        books_data.append(json.loads(line))

# Convert it into a dataframe and store it according to genre name
interaction_df_poetry = pd.DataFrame(interaction_data)
book_df_poetry = pd.DataFrame(books_data)

# Covert it into a csv file
#interaction_df_poetry.to_csv('interactions_poetry.csv')

# Drop the unnecessary columns and change the data type of the user_id column
interaction_df_poetry = interaction_df_poetry.drop(columns = ['review_id','is_read','review_text_incomplete','date_added','date_updated','read_at','started_at'])
book_df_poetry = book_df_poetry.drop(columns = ['work_id','isbn','series','asin','kindle_asin','description','link','isbn13','edition_information','title_without_series','country_code','text_reviews_count','language_code','publication_day','publication_month','is_ebook','format','publication_year','ratings_count', 'popular_shelves','publisher','similar_books'])
interaction_df_poetry['user_idInt']= interaction_df_poetry['user_id'].str.replace('\D+','').astype(float)

# Create and download the csr matrices
poetry_csr_matrix, user_dict, item_dict = create_sparse_matrix(interaction_df_poetry, user_dict, item_dict)
download_sparse_matrix('interactions_poetry', poetry_csr_matrix)

# Create a unified book_df, to store: book_id, url, image_url, authors, and title of each book. Make sure that the book_df variable doesn't go away on each iteration of coding
book_df = pd.concat([book_df, book_df_poetry], ignore_index = True)


In [ ]:
# This part is intended to convert the json files of various genres (the original data form), into .csv files.
# I have to put it in a function, so that it can be used effectively.
'''interaction_data = []
with open("/kaggle/input/goodreads-interactions-poetry/goodreads_interactions_poetry.json", 'r') as f:
    for line in f:
        interaction_data.append(json.loads(line))
        
#interaction_data1 = interaction_data[:1000000]
interaction_df_poetry = pd.DataFrame(interaction_data)
interaction_df_poetry.head(10)
display(interaction_df_poetry.describe())
interaction_df_poetry.to_csv('interactions_poetry.csv')
print(getsizeof(interaction_data1))
print("Dataframe size is", getsizeof(interaction_df1))
original_df = pd.DataFrame(columns = ['user_id', 'book_id', 'review_id', 'is_read', 'rating', 'review_text_incomplete', 'date_added', 'date_updated', 'read_at', 'started_at'])
temp_df = pd.DataFrame(columns = ['user_id', 'book_id', 'review_id', 'is_read', 'rating', 'review_text_incomplete', 'date_added', 'date_updated', 'read_at', 'started_at'])
with open("/kaggle/input/goodreads-interactions-poetry/goodreads_interactions_poetry.json", 'r') as f:
    for line in f:
        data = json.loads(line)
        original_df = pd.concat([original_df, pd.json_normalize(data)])
        #original_df.concat(json_normalize(data, '0'))
display(interaction_df1.head())
display(interaction_df1.info())
'''


## Data Visualizations

## Part 1: Neighborhood Modelling
I will try to do this part in the following two steps:
1. I will create the sparse matrix of the data and use them on distance metric code from scratch
2. Use the same sparse matrix to then evaluate the results using the implementation of similaripy

### 1. Using Sparse Matrices and simple Cosine Implementation

In [ ]:
class knn_original():
    def __init__(self, k = 20, data = None, distance_metric = 'dot_product', is_already_sparse = False, csr_item_matrix = None, alpha = 0.6):
        '''
        This class produces recommendations based on item-item similarity measures.
        The distance metrics used in this class are written from scratch, and don't have the speed benefits of the cython used by the library implementation following this.
        
        Args:
            data (DataFrame) -> Containing user_item interactions and their ratings.
            distance_metric (object) -> Defines the distance metric to be used
            k (integer) -> Represents the number of similar items that we have to produce
            csr_item_matrix (sparse matrix) -> csr matrix with items as rows and users as columns (optional)
            is_already_sparse (boolean) -> Defines if the data is already sparse.
            
        Returns:
            similarity_matrix (sparse_matrix) -> Contains the top-k similar items for each item of the data entered.
            
        '''
        _DISTANCE = ('dot_product', 'cosine','asymmetric_cosine')
        assert(distance_metric in _DISTANCE), 'The distance_metric has to be one of the following: dot_product, cosine, asymmetric_cosine' 
        self.k = k
        self.data = data
        self.csr_item_matrix = csr_item_matrix
        if is_already_sparse == False:
            self.create_csr_data()
        self.distance_metric = distance_metric
        self.alpha = alpha
        self.similarity_item_matrix = self.fit()
    
    
    
    def create_csr_data(self):
        '''
        This converts the data into the sparse matrix fromat. Again, we wouldn't need this if we end up using the supporting functions!
        '''
        # For now, I will have to clean the data first. That is, convert the columns into desired data type and remove the unecessary ones.
        self.data = self.data.drop(columns = ['review_id','is_read','review_text_incomplete','date_added','date_updated','read_at','started_at'])
        self.data['user_idInt']= self.data['user_id'].str.replace('\D+','').astype(float)
        
        # Here I'll create 'user_dict' & 'item_dict' such that: the indexes of the array would represent the "key" for the dicts.
        self.user_dict = pd.unique(self.data['user_idInt'])
        self.item_dict = pd.unique(self.data['book_id'])
        
        
        # Then, implement the csr_mechanism and save the result in self.csr_item_matrix
        rows = []
        cols = []
        data = []
        for i in range(len(self.data)):
            rows.append(self.user_dict.index(self.data.loc[i, 'user_idInt'])) # Here we creating the row values for csr matrix using the index values of the user_dict we created by mapping unique values in a dataframe
            cols.append(self.item_dict.index(self.data.loc[i, 'book_id'])) # Maybe, the index part should be in square brackets as well... Will see
            data.append(self.data.loc[i, 'rating'])
        rows = np.array(rows)
        cols = np.array(cols)
        data = np.array(data)
        self.csr_item_matrix = csr_matrix(data, (row, col))
    
        
    
    def dot_product(self, matrix):
        '''
        Finds the dot product similarity measure of the data
        
        '''
        s = matrix * matrix.T
        return self.top_k_items(s, self.k)

        
    def cosine(self, matrix):
        '''
        Finds the cosine similarity measure of the data
        
        '''
        matrix2 = matrix.copy()
        matrix2.data = np.power(matrix2.data,2)
        X = np.power(matrix2.sum(axis=1).A1,0.5)
        matrix_aux = (matrix * matrix.T).tocsr()
        r, c, v = [], [], []
        for idx1 in range(0,matrix.shape[0]):
            for idx2 in range(matrix_aux.indptr[idx1], matrix_aux.indptr[idx1+1]):
                row = idx1
                col = matrix_aux.indices[idx2]
                val = matrix_aux.data[idx2]
                r.append(row)
                c.append(col)
                v.append(val/ (X[row] * X[col]))
        s = sp.csr_matrix((v,(r,c)),shape=(matrix.shape[0],matrix.shape[0]))
        return self.top_k_items(s, self.k)

    
    def asymmetric_cosine(matrix, alpha):
        '''
        Finds the assymetrical cosine similarity measure
        
        '''
        matrix2 = matrix.copy()
        matrix2.data = np.power(matrix2.data,2)
        X = np.power(matrix2.sum(axis=1).A1,alpha)
        Y = np.power(matrix2.sum(axis=1).A1,1-alpha)
        matrix_aux = (matrix * matrix.T).tocsr()
        r, c, v = [], [], []
        for idx1 in range(0,matrix.shape[0]):
            for idx2 in range(matrix_aux.indptr[idx1], matrix_aux.indptr[idx1+1]):
                row = idx1
                col = matrix_aux.indices[idx2]
                val = matrix_aux.data[idx2]
                r.append(row)
                c.append(col)
                v.append(val/ (X[row] * Y[col]))
        s = sp.csr_matrix((v,(r,c)),shape=(matrix.shape[0],matrix.shape[0]))
        return self.top_k_items(s, self.k)
        
        
    def top_k_items(self, X, k):
        '''
        
        '''
        X = X.tocsr()
        r, c, d = [], [], []
        for i in range(X.shape[0]):
            data = X.data[X.indptr[i]:X.indptr[i+1]]
            topk = min(k, data.shape[0])
            indices = X.indices[X.indptr[i]:X.indptr[i+1]]
            topk_idx = np.argpartition(data, -topk)[-topk:]
            data = data[topk_idx]
            indices = indices[topk_idx]
            r += np.full(topk, i).tolist()
            c += indices.tolist()
            d += data.tolist()
        return sp.csr_matrix((d, (r, c)), shape=X.shape)
        
        
        
    def fit_model(self):
        '''
        The central function that calls the necesary functions to produce top-k recommendations
        
        '''
        self.old_csr_item_matrix = self.csr_item_matrix.copy()
        if self.distance_metric == 'dot_product':
            similarity_scores = self.dot_product(self.csr_item_matrix)
        elif self.distance_metric == 'cosine':
            similarity_scores = self.cosine(self.csr_item_matrix)
        else:
            similarity_scores = self.asymmetric_cosine(self.csr_item_matrix, self.alpha)
        return similarity_scores
        
    
    def make_recommendations(self,user_id):
        '''
        
        '''
        self.user_id = user_id
        self.user_id = self.user_id.str.replace('\D+','').astype(float)
        user_id_to_check = self.user_dict.index(self.user_id)
        matrix = self.old_csr_item_matrix.T.tocsr()
        indices = matrix.indices
        indptr = matrix.indptr
        data = matrix.data
        book_list = []
        for i in range(indptr[user_id_to_check], indptr[user_id_to_check + 1]):
            book_list.append(data[i].astype(int))
        indices = self.similarity_scores.indices
        indptr = self.similairity_scores.indptr
        data = self.similarity_scores.data
        positional_increment = 0.05 # This defines the extra amount of importance added with better position in the list. So, if the last element (of a 10 item recommendation) is has a value of 1, the first element would have a value of 1 + (0.05 * 10)
        recommendation_df = pd.DataFrame(columns = ['book_id', 'positional_value'])
        for book in book_list:
            top_k_recommendations = []
            for i in range(indptr[book], indptr[book + 1]):
                top_k_recommendations.append(data[i].astype(int))
            start_value = 1
            m = len(top_k_recommendations)
            for i in range(m - 1, -1, -1):
                book_id = top_k_recommendations[i]
                position_value = start_value + (m - (i+1))*positional_increment
                book_df = pd.DataFrame([book_id, positional_value], columns = ['book_id','positional_value'])
                recommendation_df = pd.concat([recommendation_df, book_df], axis = 1)
        recommendation_df = recommendation_df.groupby('book_id').sum()
        recommendation_df = recommendation_df.sort_values('positional_value')[:self.k]
        self.recommendations = np.array(recommendation_df.loc[:,'book_id'])

### 2. Using Sparse Matrices and Similaripy Implementation

In [ ]:
# I can even include the train test split and evaluate the model's efficacy using RMSE, but for that, I will have to figure out how to extract similarity measures from 
# the implicit library, and then transform those similarity measures using 
class knn_similaripy():
    def __init__(self, normalization_type = None, tf_type= 'sqrt', idf_type = 'bm25', distance_metric = 'cosine',
                 user_id = None, data = None, csr_item_matrix = None, k = 10, is_already_sparse = False,
                 create_ratings = False):
        '''
        This class produces recommendations based on Nearest Neighbour mechanism. We will use the KNN implementation of implicit library to perform this task.
        
        Args:
            data (DataFrame) -> Actual DataFrame containing three columns - 'book_id', 'user_id', 'rating'
            user_id (Int) -> User Id of the user for which we want to make the recommendations
            k (int) -> Number of recommendations we want to produce
        
        Returns:
            recommendations (iterable) -> A list/array of item_id, of the books being recommended.
        
        '''
        _NORMALIZATIONS = ('l1', 'l2', 'max')
        _TF = ('binary', 'raw', 'sqrt', 'freq', 'log')
        _IDF = ('unary', 'base', 'smooth', 'prob', 'bm25', 'bm25plus')
        _DISTANCE_METRIC = ('dot_product', 'cosine', 'assymetric_cosine','jaccard','dice','tversky','p3alpha','rp3beta')
        assert(normalization_type in _NORMALIZATIONS), 'Normalization functions can only be l1, l2, max'
        assert(tf_type in _TF), 'Term Frequency functions can only be binary, raw, sqrt, freq, log'
        assert(idf_type in _IDF), 'Inverse Document Frequency functions can only be one of unary, base, smooth, prob, bm25, bm25plus'
        assert(distance_metric in _DISTANCE_METRIC), 'Distance Metric must be one of the many prescribed by Similaripy'
        self.user_id = user_id
        self.data = data
        self.k = k
        self.csr_item_matrix = csr_item_matrix # This will house the sparse matrix of the original data (if passed data isn't csr_matrix already)
        self.is_already_sparse = is_already_sparse
        if not self.is_already_sparse and data != None:
            self.create_csr_data()
        self.normalization_type = normalization_type
        self.tf_type = tf_type
        self.idf_type = idf_type
        self.distance_metric = distance_metric
        self.recommendations = None
        self.ratings_matrix = None # Here I will store the predicted ratings for all user item pairs. Special care will have to be taken to ensure that this does not excede memory.
        self.create_ratings = create_ratings
        self.train_data = None # Here I will store the train and test data, respectively
        self.test_data = None
        if self.create_ratings:
            self.test_train_split()
        self.similarity_scores = self.fit_model()
        
        
        
    def create_csr_data(self):
        '''
        This function converts the original dataframe into a sparse matrix.
        
        '''
        # For now, I will have to clean the data first. That is, convert the columns into desired data type and remove the unecessary ones.
        self.data = self.data.drop(columns = ['review_id','is_read','review_text_incomplete','date_added','date_updated','read_at','started_at'])
        self.data['user_idInt']= self.data['user_id'].str.replace('\D+','').astype(float)
        
        # Here I'll create 'user_dict' & 'item_dict' such that: the indexes of the array would represent the "key" for the dicts.
        self.user_dict = pd.unique(self.data['user_idInt'])
        self.item_dict = pd.unique(self.data['book_id'])
        
        
        # Then, implement the csr_mechanism and save the result in self.csr_item_matrix
        rows = []
        cols = []
        data = []
        for i in range(len(self.data)):
            rows.append(self.user_dict.index(self.data.loc[i, 'user_idInt'])) # Here we creating the row values for csr matrix using the index values of the user_dict we created by mapping unique values in a dataframe
            cols.append(self.item_dict.index(self.data.loc[i, 'book_id'])) # Maybe, the index part should be in square brackets as well... Will see
            data.append(self.data.loc[i, 'rating'])
        rows = np.array(rows)
        cols = np.array(cols)
        data = np.array(data)
        self.csr_item_matrix = csr_matrix(data, (row, col))
    
    
    def fit_model(self):
        '''
        This function fits the Similaripy model to the given data, and creates the 'similarity_matrix' for all the items.
        
        First I'll normalize the data and tranform with the appropriate tf_idf transform.
        Then, I'll feed the data with the specific distance metric and produce the similarity matrix
        
        
        '''
        self.old_csr_item_matrix = self.csr_item_matrix.copy()
        # Normalization part
        if self.normalization_type == None:
            continue
        else:
            self.csr_item_matrix = normalize(self.csr_item_matrix, axis = 1, norm = self.normalization_type)
            
        # Tf_IDf part
        if self.tf_type == None and self.idf_type == None:
            continue
        elif self.idf_type == 'bm25':
            self.csr_item_matrix = bm25(self.csr_item_matrix, tf_mode = self.tf_type, idf_mode = self.idf_type)
        elif self.idf_type == 'bm25plus':
            self.csr_item_matrix = bm25plus(self.csr_item_matrix, tf_mode = self.tf_type, idf_mode = self.idf_type)
        else:
            self.csr_item_matrix = tfidf(self.csr_item_matrix, tf_mode = self.tf_type, idf_mode = self.idf_type)
        
        # Fitting the model (!!!Still have to figure out if this returns the a csr matrix with top k items for each item or not!!!)
        if self.distance_metric == 'dot_product':
            model = sim.dot_product(self.csr_item_matrix, k = self.k)
        elif self.distance_metric == 'cosine':
            model = sim.cosine(self.csr_item_matrix, k = self.k)
        elif self.distance_metric == 'assymetric_cosine':
            model = sim.assymetric_cosine(self.csr_item_matrix, alpha = 0.2, k = self.k)
        elif self.distance_metric == 'jaccard':
            model = sim.jaccard(self.csr_item_matrix, k = self.k)
        elif self.distance_metric == 'dice':
            model = sim.dice(self.csr_item_matrix, k = self.k)
        elif self.distance_metric == 'tversky':
            model = sim.tversky(self.csr_item_matrix, alpha = 0.8, beta = 0.4, k = self.k)
        elif self.distance_metric == 'p3alpha':
            model = sim.p3alpha(self.csr_item_matrix, alpha = 0.8, k = self.k)
        else:
            model = sim.rp3beta(self.csr_item_matrix, alpha = 0.8, beta = 0.4, k = self.k)
        
        return model # Here I am assuming that the model creates similarity scores of top-k items for each item in self.similarity_scores (which it most probably does!)
    
    
    def create_recommendations(self, user_id = 1, positional_increment = 0.05):
        '''
        This function covers the utility aspect of the recommendation systems, by making book recommendations for the given user_id, by making sure that the user history is 
        taken into account.
        
        Since, this model produces similarity scores of the items, producing recommendations for a specific user has to do the following task:
        It produces top-K similar books for each of the items that the user has rated. It then produces a normalized score of those recommendations (based on user rating).
        The top-K books thus produced are then recommended to the user.
        
        Here, I will have to decide if I want to implement that normalization approach where the baseline ratings are removed from the rating_score being generated,
        to account for the user bias (some users are prone to giving too good or too bad ratings)
        
        Args:
            user_id (object/int) -> User_id either in form given in the dataset or in integer form
            positional_increment (float) -> Defines the extra bit of importance we give to having a single position advantage in recommendations
        
        '''
        self.user_id = user_id
        self.user_id = self.user_id.str.replace('\D+','').astype(float)
        user_id_to_check = self.user_dict.index(self.user_id)
        matrix = self.old_csr_item_matrix.T.tocsr()
        indices = matrix.indices
        indptr = matrix.indptr
        data = matrix.data
        book_list = []
        for i in range(indptr[user_id_to_check], indptr[user_id_to_check + 1]):
            book_list.append(data[i].astype(int))
        indices = self.similarity_scores.indices
        indptr = self.similairity_scores.indptr
        data = self.similarity_scores.data
        positional_increment = 0.05 # This defines the extra amount of importance added with better position in the list. So, if the last element (of a 10 item recommendation) is has a value of 1, the first element would have a value of 1 + (0.05 * 10)
        recommendation_df = pd.DataFrame(columns = ['book_id', 'positional_value'])
        for book in book_list:
            top_k_recommendations = []
            for i in range(indptr[book], indptr[book + 1]):
                top_k_recommendations.append(data[i].astype(int))
            start_value = 1
            m = len(top_k_recommendations)
            for i in range(m - 1, -1, -1):
                book_id = top_k_recommendations[i]
                position_value = start_value + (m - (i+1))*positional_increment
                book_df = pd.DataFrame([book_id, positional_value], columns = ['book_id','positional_value'])
                recommendation_df = pd.concat([recommendation_df, book_df], axis = 1)
        recommendation_df = recommendation_df.groupby('book_id').sum()
        recommendation_df = recommendation_df.sort_values('positional_value')[:self.k]
        self.recommendations = np.array(recommendation_df.loc[:,'book_id'])
    
    
    def test_train_split(self):
        '''
        This function will only be used if we are gonna produce ratings as well, along with recommendations. The main problem with this approach would be that we can 
        only create ratings for the items that do fall into the 'top-K' similar items for each item, and there are high chances that the user might not have actually rated
        any/all of the books that we are creating the ratings for. 
        So for a sufficiently large enough test set, the number actual test instances that we can compare our model's prediction capability would be actually quite small.
        
        '''
        
        
        
        
        
        
    def create_ratings(self):
        '''
        This function will convert the similarity values, (derived from fit_model() function) using the approach defined in 'Neigborhood meets Factorization' paper
        
        
        '''
        
        
        
        
        
        
    def efficacy_score(self):
        '''
        Using the test train split, and after creating the ratings, this function will produce the error metric on recommendations using RMSE score as the deciding metric.
        This would allow us to compare various models amongst each other
        
        '''
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    

In [ ]:
poetry_nn_df1 = interactions_df.copy()


##### Shortcomings still remaining:
* Point 1
* Point 2

## Part 2: Matrix Factorization Method

### Surprise Implementation

#### 1. Creating Data

In [17]:
# Getting the data into the format Surprise Requires it in
reader = Reader(rating_scale=(1, 5))
data_surprise = Dataset.load_from_df(interactions_df[['user_idInt', 'book_id', 'rating']], reader)
# Creating train and test splits, with 10 percent of data being saved for 
training_data, testing_data = train_test_split(data_surprise, test_size=.10)



#### 2. Defining Models

In [18]:
algo1 = SVD(n_factors = 20, n_epochs = 10, verbose = True)
algo1.fit(training_data)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9


In [46]:
# Here we try to increase the number of factors to 50, and number of epochs to 30
algo2 = SVD(n_factors = 50, n_epochs = 30, verbose = True)
algo2.fit(training_data)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29


In [48]:
algo3 = SVD(n_factors = 100, n_epochs = 30, lr_all = 0.0075, reg_all = 0.03, verbose = True)
algo3.fit(training_data)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29


#### 3. Evaluating RMSE scores of test set

In [45]:
predictions = algo1.test(testing_data)
print(predictions[0:5])
predicted_values = [prediction[3] for prediction in predictions]
actual_values = [prediction[2] for prediction in predictions]
score = mean_squared_error(predicted_values, actual_values)
print('Mean Squared Error for Algo1 is ', score)


[Prediction(uid=7.990865635606991e+18, iid=5365890, r_ui=5.0, est=4.21245686299496, details={'was_impossible': False}), Prediction(uid=3334632421182.0, iid=23513349, r_ui=5.0, est=4.039836109510277, details={'was_impossible': False}), Prediction(uid=5.117867507246729e+23, iid=135914, r_ui=2.0, est=4.434237960252834, details={'was_impossible': False}), Prediction(uid=3.394076697144432e+18, iid=1371, r_ui=5.0, est=3.5618849103250705, details={'was_impossible': False}), Prediction(uid=1.9102173198936143e+18, iid=5865595, r_ui=4.0, est=3.69674387896062, details={'was_impossible': False})]
Mean Squared Error is  0.7599449695931739


In [47]:
predictions = algo2.test(testing_data)
print(predictions[0:5])
predicted_values = [prediction[3] for prediction in predictions]
actual_values = [prediction[2] for prediction in predictions]
score = mean_squared_error(predicted_values, actual_values)
print('Mean Squared Error for Algo1 is ', score)

[Prediction(uid=7.990865635606991e+18, iid=5365890, r_ui=5.0, est=4.255335465276854, details={'was_impossible': False}), Prediction(uid=3334632421182.0, iid=23513349, r_ui=5.0, est=3.4619858989446866, details={'was_impossible': False}), Prediction(uid=5.117867507246729e+23, iid=135914, r_ui=2.0, est=4.344592732722931, details={'was_impossible': False}), Prediction(uid=3.394076697144432e+18, iid=1371, r_ui=5.0, est=2.9907983023522045, details={'was_impossible': False}), Prediction(uid=1.9102173198936143e+18, iid=5865595, r_ui=4.0, est=4.115873258407759, details={'was_impossible': False})]
Mean Squared Error for Algo1 is  0.7430516785096469


In [49]:
predictions = algo3.test(testing_data)
print(predictions[0:5])
predicted_values = [prediction[3] for prediction in predictions]
actual_values = [prediction[2] for prediction in predictions]
score = mean_squared_error(predicted_values, actual_values)
print('Mean Squared Error for Algo1 is ', score)

[Prediction(uid=7.990865635606991e+18, iid=5365890, r_ui=5.0, est=4.226096772677253, details={'was_impossible': False}), Prediction(uid=3334632421182.0, iid=23513349, r_ui=5.0, est=3.8330093413615964, details={'was_impossible': False}), Prediction(uid=5.117867507246729e+23, iid=135914, r_ui=2.0, est=4.3549111890341585, details={'was_impossible': False}), Prediction(uid=3.394076697144432e+18, iid=1371, r_ui=5.0, est=3.3684022550126596, details={'was_impossible': False}), Prediction(uid=1.9102173198936143e+18, iid=5865595, r_ui=4.0, est=3.907478088763133, details={'was_impossible': False})]
Mean Squared Error for Algo1 is  0.7451192479396285


##### Result Evaluation and comparison of various models
* Increasing the number of factors and number of epochs helped improve the MSE score.
* Increasing the learning rate and relaxing the regularization rate actually detoriated the performance of the algorithm a bit

After training the model with the Surprise implementation of SVD, we could produce the recommendations using the following metric:
* Simple rating values of all the items for a given user, and then print the top 10 from the sorted list
* Rating values of only the top-k items recommended by the item similarity algorithm, and printing them out. This would allow us to compare the efficacy of different models.

##### Shortcomings still remaining:
* It does not use sparse matrices. Therefore making it tough for working on the big files I have
* While it has a 'dump' function for downloading the trained model and loading it later, it does not have iterative training. Therefore, it will have to train the model from scratch each time
* It does not have an option for trying out ALS implementation of MF.
* The SVD implementation of Surprise cannot accomodate implicit data, we will try to incorporate that in the next section using SVD++

## Part 3: Weighted Regularized Matrix Factorization

## Part 4: Neural Collaborative Filtering (Optional)